In [6]:
from arcgis.gis import GIS
from arcgis.geometry import Envelope, Point
from arcgis.features import Feature, FeatureSet
from datetime import datetime
from data_engineering.utils import fetch_charging_stations, fetch_hotcold_features, fetch_hottest_features_by_extent, fetch_traffic_accidents, get_hotcold_layer, get_live_traffic_item
from urban_traffic.utils import create_map, create_traffic_map, fetch_traffic_data, filter_commute_cars, generate_car_renderer, prepare_traffic, read_traffic_features
import os
import pandas as pd

### Additional dependencies
We load the necessary variables using `dotenv`.  
Please, ensure that you setup before: `pip install python-dotenv`  
Furthermore, put a `.env` file into the root of this repository.

In [52]:
from dotenv import load_dotenv

def load_vars():
    load_dotenv(dotenv_path="../../../../.env", override=True)
    return os.getenv("ARCGIS_API_KEY"), os.getenv("TRAFFIC_DATA_FILE"), os.getenv("TRAFFIC_FEATURES"), os.getenv("NETWORK_DATASET"), os.getenv("ELECTRIC_CAR_FEATURES")

In [53]:
api_key, traffic_data_filepath, traffic_features_filepath, network_dataset_filepath, electric_car_features_filepath = load_vars()

### Connect to your GIS portal

In [54]:
gis = GIS(api_key=api_key)

### Use a map for the area of interest

In [55]:
traffic_map_view = create_traffic_map(gis)
traffic_map_view

Map(center=[6465801.0751674855, 962955.6795531422], extent={'xmin': 962882.1493646716, 'ymin': 6465580.1136933…

In [114]:
map_view = create_map(gis, location="Frankenallee 355, Frankfurt am Main, Germany")
map_view

Map(center=[6463970.497258207, 959241.6442823887], extent={'xmin': 959131.7990409209, 'ymin': 6463772.48065509…

### Spatial filter using the area of interest

In [115]:
distance_in_meters = 150
lon, lat = map_view.center[:2]
traffic_features = read_traffic_features(traffic_features_filepath, lon=lon, lat=lat, meters=distance_in_meters)
traffic_features[["trip", "person", "vehicle_type"]]

,trip,person,vehicle_type
0,11661,7271,Bike
1,11661,7271,Bike
2,11661,7271,Bike
3,11661,7271,Bike
4,11661,7271,Bike
...,...,...,...
7969,1373,858,Bike
7970,1373,858,Bike
7971,1373,858,Bike
7972,1373,858,Bike


## Ingestion & Data Engineering

In [116]:
traffic_features.dtypes

OBJECTID           int64
SHAPE           geometry
trip               int64
person             int64
vehicle_type      object
trip_time         object
dtype: object

In [117]:
prepared_traffic_features = prepare_traffic(traffic_features.copy())
prepared_traffic_features

,OBJECTID,SHAPE,trip,person,hour,minute,second,bike,car,pedestrian
0,75210,"{""x"": 8.615455619000045, ""y"": 50.1027864670000...",11661,7271,1,52,28,1,0,0
1,75233,"{""x"": 8.615466635000075, ""y"": 50.1027515590000...",11661,7271,1,52,29,1,0,0
2,75258,"{""x"": 8.61547765000006, ""y"": 50.10271665200002...",11661,7271,1,52,30,1,0,0
3,75282,"{""x"": 8.615488666000033, ""y"": 50.1026817450000...",11661,7271,1,52,31,1,0,0
4,75306,"{""x"": 8.615499682000063, ""y"": 50.1026468380000...",11661,7271,1,52,32,1,0,0
...,...,...,...,...,...,...,...,...,...,...
7969,17228677,"{""x"": 8.618661074000045, ""y"": 50.1012395350000...",1373,858,23,32,58,1,0,0
7970,17228705,"{""x"": 8.618668207000042, ""y"": 50.1012061070000...",1373,858,23,32,59,1,0,0
7971,17228731,"{""x"": 8.618675339000049, ""y"": 50.1011726770000...",1373,858,23,33,0,1,0,0
7972,17228758,"{""x"": 8.618682471000056, ""y"": 50.1011392490000...",1373,858,23,33,1,1,0,0


In [118]:
prepared_traffic_features.dtypes

OBJECTID         int64
SHAPE         geometry
trip             int64
person           int64
hour             int32
minute           int32
second           int32
bike             int32
car              int32
pedestrian       int32
dtype: object

### Visualize commute traffic

In [119]:
commute_cars = filter_commute_cars(prepared_traffic_features.copy())
commute_cars

,OBJECTID,SHAPE,trip,person,hour,minute,second,bike,car,pedestrian
1294,2423183,"{""x"": 8.618313765000039, ""y"": 50.1029611390000...",4465,2771,8,4,8,0,1,0
1295,2423570,"{""x"": 8.618321629000036, ""y"": 50.1028414760000...",4465,2771,8,4,9,0,1,0
1296,2424352,"{""x"": 8.618350565000071, ""y"": 50.1027179640000...",4465,2771,8,4,10,0,1,0
1297,2424795,"{""x"": 8.618323860000032, ""y"": 50.1029196350000...",2228,1378,8,4,11,0,1,0
1298,2424953,"{""x"": 8.61837866600007, ""y"": 50.10259437100006...",4465,2771,8,4,11,0,1,0
...,...,...,...,...,...,...,...,...,...,...
3258,6734311,"{""x"": 8.615983551000056, ""y"": 50.1012082780000...",460,292,9,50,35,0,1,0
3259,6735153,"{""x"": 8.616021930000045, ""y"": 50.1010858360000...",460,292,9,50,36,0,1,0
3260,6735842,"{""x"": 8.61606094900003, ""y"": 50.10096347300003...",460,292,9,50,37,0,1,0
3261,6736537,"{""x"": 8.61609997000005, ""y"": 50.10084111000003...",460,292,9,50,38,0,1,0


In [120]:
commute_map_view = create_map(gis)
commute_cars.spatial.plot(commute_map_view, renderer=generate_car_renderer())
commute_map_view.zoom_to_layer(commute_cars)
commute_map_view

Map(center=[6465801.0751674855, 962955.6795531422], extent={'xmin': 959103.9242357736, 'ymin': 6463736.8591492…

## Advanced Spatial Analysis

In [121]:
hotcold_spots, drawing_info = fetch_hotcold_features(gis, commute_cars)
commute_map_view.content.add(hotcold_spots, drawing_info=drawing_info, options={"opacity": 0.3})

In [122]:
charging_stations = fetch_charging_stations(gis, extent=Envelope(commute_map_view.extent))
charging_stations

,OBJECTID,Ladeeinrichtungs_ID,Betreiber,Anzeigename__Karte_,Status,Art_der_Ladeeinrichtung,Anzahl_Ladepunkte,Nennleistung_Ladeeinrichtung__k,Inbetriebnahmedatum,Straße,...,Public_Key4,Steckertypen5,Nennleistung_Stecker5,EVSE_ID5,Public_Key5,Steckertypen6,Nennleistung_Stecker6,EVSE_ID6,Public_Key6,SHAPE
0,42679,1111949.0,On Charge GmbH,<NA>,In Betrieb,Normalladeeinrichtung,2.0,27.0,2023-08-29,Frankenallee,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"{""x"": 959243.1691000015, ""y"": 6463961.64850000..."
1,43006,1068595.0,Volkswagen Automobile Frankfurt GmbH,Automobile Frankfurt GmbH,In Betrieb,Schnellladeeinrichtung,1.0,75.0,2020-07-31,Mainzer Landstr.,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"{""x"": 960018.9545999989, ""y"": 6463911.66589999..."


In [123]:
import arcpy
from arcpy.da import SearchCursor
from arcpy.management import CopyFeatures
from arcpy.na import AddLocations, GetNAClassNames, Solve

In [124]:
arcpy.env.overwriteOutput = True

In [125]:
analysis_layer_name = "ClosestChargingStation"
layer_result = arcpy.na.MakeClosestFacilityAnalysisLayer(network_dataset_filepath, analysis_layer_name)
layer_result

<Result 'GPLYR_{55F07F33-53FE-4B09-9251-F39331D2FD82}'>

In [126]:
analysis_layer = layer_result.getOutput(0)
analysis_class_names = GetNAClassNames(analysis_layer)
analysis_class_names

{'Facilities': 'Facilities',
 'Incidents': 'Incidents',
 'Barriers': 'Point Barriers',
 'CFRoutes': 'Routes',
 'PolylineBarriers': 'Line Barriers',
 'PolygonBarriers': 'Polygon Barriers'}

In [127]:
input_facilities = "memory/facilities"
charging_stations.spatial.to_featureclass(input_facilities)

'memory\\facilities'

In [128]:
AddLocations(analysis_layer, analysis_class_names["Facilities"], input_facilities)

<Result 'GPLYR_{92547CA4-11B2-4300-A786-071D26585621}'>

In [129]:
lon, lat = commute_map_view.center[:2]
input_location = Point({
    "x": lon,
    "y": lat,
    "spatialReference": {"wkid": 4326}
})

input_feature = Feature(input_location, {
    "trip_time": datetime.now().isoformat()
})

input_featureset = FeatureSet([input_feature])
input_featureset.sdf

,trip_time,OBJECTID,SHAPE
0,2025-11-13T17:11:18.686840,1,"{""x"": 8.617282283000065, ""y"": 50.1018399561209..."


In [130]:
latest_positions = pd.DataFrame.spatial.from_featureclass(electric_car_features_filepath)
latest_positions

,OBJECTID,SHAPE
0,1,"{""x"": 944663.6695000008, ""y"": 6460319.02390000..."
1,2,"{""x"": 971470.192400001, ""y"": 6457397.931100003..."
2,3,"{""x"": 977864.9254000001, ""y"": 6470463.21670000..."
3,4,"{""x"": 972219.3685999997, ""y"": 6480431.19870000..."
4,5,"{""x"": 943752.0808000006, ""y"": 6475590.21429999..."


In [131]:
input_incidents = "memory/incidents"
#input_featureset.sdf.spatial.to_featureclass(input_incidents)
latest_positions.spatial.to_featureclass(input_incidents)
AddLocations(analysis_layer, analysis_class_names["Incidents"], input_incidents)

<Result 'GPLYR_{92547CA4-11B2-4300-A786-071D26585621}'>

In [132]:
solve_result = Solve(analysis_layer)
solve_result

<Result 'GPLYR_{92547CA4-11B2-4300-A786-071D26585621}'>

In [133]:
result_layer = solve_result.getOutput(0)
result_layer

In [134]:
with SearchCursor(result_layer, "*") as cursor:
    for row in cursor:
        print(row)

(1, (959243.1691000015, 6463961.648500003), 'Location 1', 1, 40562499, 0.4563989736266256, 1, 0, 0, 959241.5386321871, 6463972.870927805, 0.0, 11.340251807983583, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, None, None, None, None, None, None, None, None)
(2, (960018.9545999989, 6463911.665899999), 'Location 2', 1, 40562543, 0.6012595788641886, 2, 0, 0, 959974.7608000003, 6463948.805699997, 0.0, 57.72743457100831, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, None, None, None, None, None, None, None, None)


In [135]:
output_routes = "memory/routes"
CopyFeatures(analysis_class_names["CFRoutes"], output_routes)

<Result 'memory\\routes'>

In [136]:
charging_routes = pd.DataFrame.spatial.from_featureclass(output_routes)
charging_routes

,OBJECTID,FacilityID,FacilityRank,Name,IncidentCurbApproach,FacilityCurbApproach,IncidentID,StartTime,EndTime,StartTimeUTC,EndTimeUTC,Total_Minutes,Total_TravelTime,Total_Miles,Total_Kilometers,Total_TimeAt1KPH,Total_WalkTime,Total_TruckMinutes,Total_TruckTravelTime,SHAPE
0,1,1,1,Location 1 - Location 1,1,2,1,NaT,NaT,NaT,NaT,<NA>,10.816478,<NA>,11.3742,<NA>,<NA>,<NA>,<NA>,"{""hasM"": true, ""paths"": [[[944661.0929999985, ..."
1,2,1,1,Location 2 - Location 1,2,2,2,NaT,NaT,NaT,NaT,<NA>,17.015653,<NA>,22.963932,<NA>,<NA>,<NA>,<NA>,"{""hasM"": true, ""paths"": [[[971478.6953999996, ..."
2,3,1,1,Location 3 - Location 1,1,2,3,NaT,NaT,NaT,NaT,<NA>,25.482444,<NA>,29.851264,<NA>,<NA>,<NA>,<NA>,"{""hasM"": true, ""paths"": [[[977871.9464000016, ..."
3,4,1,1,Location 4 - Location 1,2,2,4,NaT,NaT,NaT,NaT,<NA>,19.50582,<NA>,28.044713,<NA>,<NA>,<NA>,<NA>,"{""hasM"": true, ""paths"": [[[972217.7173000015, ..."
4,5,1,1,Location 5 - Location 1,1,2,5,NaT,NaT,NaT,NaT,<NA>,16.256045,<NA>,20.073187,<NA>,<NA>,<NA>,<NA>,"{""hasM"": true, ""paths"": [[[943749.5025000013, ..."


In [137]:
from arcgis.map.renderers import SimpleRenderer
from arcgis.map.symbols import SimpleMarkerSymbolEsriSMS, SimpleMarkerSymbolStyle, SimpleLineSymbolEsriSLS, SimpleLineSymbolStyle


def generate_routes_renderer():
    return SimpleRenderer(
        symbol=SimpleLineSymbolEsriSLS(
            color=[255, 155, 155, 155],
            width=7.5,
            style=SimpleLineSymbolStyle.esri_sls_solid.value,
        )
    )

In [138]:
charging_map = create_map(gis)
latest_positions.spatial.plot(charging_map, renderer=generate_car_renderer())
charging_stations.spatial.plot(charging_map, renderer=generate_car_renderer())
charging_routes.spatial.plot(charging_map, renderer=generate_routes_renderer())
charging_map.zoom = 12
charging_map

Map(center=[6465801.0751674855, 962955.6795531422], extent={'xmin': 962882.1493646716, 'ymin': 6465580.1136933…

In [139]:
hottest_spots, drawing_info = fetch_hottest_features_by_extent(gis, extent=Envelope(charging_map.extent))
charging_map.content.add(hottest_spots, drawing_info=drawing_info, options={"opacity": 0.3})

In [140]:
input_barriers = "memory/barriers"
hottest_spots.sdf.spatial.to_featureclass(input_barriers)
AddLocations(analysis_layer, analysis_class_names["PolygonBarriers"], input_barriers)

<Result 'GPLYR_{92547CA4-11B2-4300-A786-071D26585621}'>

In [143]:
# Solve again
solve_result = Solve(analysis_layer)

# Extract the solved routes
output_routes = "memory/routes"
CopyFeatures(analysis_class_names["CFRoutes"], output_routes)
charging_routes = pd.DataFrame.spatial.from_featureclass(output_routes)

# Put it on a map
charging_map = create_map(gis)
charging_stations.spatial.plot(charging_map, renderer=generate_car_renderer())
charging_routes.spatial.plot(charging_map, renderer=generate_routes_renderer())
charging_map.content.add(hottest_spots, drawing_info=drawing_info, options={"opacity": 0.3})
charging_map.zoom = 12
charging_map

Map(center=[6465801.0751674855, 962955.6795531422], extent={'xmin': 962882.1493646716, 'ymin': 6465580.1136933…